In [ ]:
# Upgrade pip to avoid dependency resolution issues
!pip install --upgrade pip

# Core utilities
!pip install python-dotenv tqdm

# LangChain modular packages (post-0.1.x architecture)
!pip install \
    langchain-core \
    langchain-community \
    langchain-groq

# Vector store backend
!pip install chromadb

# Embeddings dependencies
!pip install sentence-transformers huggingface-hub

# Optional but commonly required by Chroma + LangChain
!pip install pydantic typing-extensions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s  0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [langchain-community]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import json
from tqdm import tqdm
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive


 Agentic-RAG-Chatbot  'Colab Notebooks'   Colab_Notebooks   extra


In [ ]:
%cd "/content/drive/MyDrive/Agentic-RAG-Chatbot"


/content/drive/MyDrive/Agentic-RAG-Chatbot


In [ ]:
!ls

app.py		   evaluation_results_1_50.json  notebook.ipynb
Chatbot.ipynb	   main.py			 __pycache__
dataset_loader.py  metrics.py			 requirements.txt


In [ ]:
import dataset_loader

In [ ]:
# Import existing modules
from dataset_loader import load_squad_v2, prepare_contexts_for_rag
from metrics import evaluate_batch, evaluate_unanswerable

In [ ]:
# CONFIGURATION
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "openai/gpt-oss-20b"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
K_RETRIEVED = 3
MAX_SAMPLES = 50  # Set to None to use all samples

In [ ]:
# RAG SETUP
def format_docs(docs):
    """Format retrieved documents for context"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        formatted.append(f"[Source {i}]\n{doc.page_content}")
    return "\n\n".join(formatted)


def setup_rag_system(contexts, persist_directory="./chroma_db"):
    """Setup complete RAG system with retriever and LLM"""

    # Embeddings
    embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

    # Check if index exists
    if os.path.exists(persist_directory) and os.listdir(persist_directory):
      print("Loading existing vector store from {persist_directory}...")

      # Load existing index
      vectorestore= Chroma(persist_directory= persist_directory, embedding_function=embedding_model)
      print("Loaded existing index!")
    else:
      print("Creating new vector store and saving to {persist_directory}...")

    # Create vector store
    vectorstore = Chroma.from_documents(
        documents=contexts,
        embedding=embedding_model,
        persist_directory= persist_directory
    )
    print ("Created and saved new index!")

    # Retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": K_RETRIEVED}
    )

    # LLM
    llm = ChatGroq(
        model=LLM_MODEL,
        groq_api_key=GROQ_API_KEY,
        temperature=0
    )

    # Prompt
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a question-answering assistant. "
            "Answer the question using ONLY the provided context. "
            "If the answer cannot be found in the context, respond with 'I don't know' or 'The answer is not available in the provided context.'"
        ),
        (
            "human",
            "Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"
        )
    ])

    # RAG Chain
    rag_chain = (
        {
            "docs": itemgetter("question") | retriever,
            "question": itemgetter("question"),
        }
        | RunnableLambda(lambda x: {
            "question": x["question"],
            "context": format_docs(x["docs"]),
        })
        | RunnableLambda(lambda x: {
            "answer": (
                prompt
                | llm
                | StrOutputParser()
            ).invoke({
                "question": x["question"],
                "context": x["context"],
            })
        })
    )

    print("RAG system ready!")
    return rag_chain

In [ ]:
# EVALUATION
def evaluate_on_squad(rag_chain, examples, max_samples=MAX_SAMPLES):
    """Evaluate RAG system """
    if max_samples:
        examples = examples[:max_samples]
        print(f"Using {max_samples} samples for evaluation")

    predictions = []
    ground_truths_list = []
    is_impossible_list = []

    print(f"\nEvaluating on {len(examples)} examples...")

    for example in tqdm(examples, desc="Processing"):
        try:
            # Get prediction from RAG system
            result = rag_chain.invoke({"question": example['question']})
            prediction = result["answer"]
            predictions.append(prediction)

            # Get ground truth answers
            ground_truths = example['answers']['text']
            ground_truths_list.append(ground_truths)

            # Track if unanswerable
            is_impossible_list.append(example['is_impossible'])
        except Exception as e:
            print(f"Error processing example {example['id']}: {e}")
            predictions.append("")
            ground_truths_list.append(example['answers']['text'])
            is_impossible_list.append(example['is_impossible'])

    # Evaluate using metrics.py
    metrics = evaluate_batch(predictions, ground_truths_list)
    unanswerable_metrics = evaluate_unanswerable(predictions, is_impossible_list)

    # Combine results
    results = {
        **metrics,
        **unanswerable_metrics,
        'total_samples': len(examples),
        'answerable_samples': sum(1 - x for x in is_impossible_list),
        'unanswerable_samples': sum(is_impossible_list)
    }

    return results, predictions

In [ ]:
def main():
    """Main evaluation pipeline"""
    print("SQuAD v2 RAG Evaluation Pipeline")

    # Step 1: Load SQuAD v2 dataset (using your dataset_loader.py)
    print("\nStep 1: Loading dataset...")
    examples = load_squad_v2(split='validation')
    print(f"Loaded {len(examples)} examples")

    # Step 2: Prepare contexts for RAG (using your dataset_loader.py)
    print("\nStep 2: Preparing contexts...")
    contexts = prepare_contexts_for_rag(
        examples,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    print(f"Created {len(contexts)} document chunks")

    # Step 3: Setup RAG system
    print("\nStep 3: Setting up RAG system...")
    rag_chain = setup_rag_system(contexts)

    # Step 4: Run evaluation
    print("\nStep 4: Running evaluation...")
    results, predictions = evaluate_on_squad(rag_chain, examples, MAX_SAMPLES)

    # Step 5: Display results
    print("\n" + "="*60)
    print("EVALUATION RESULTS")
    print("="*60)
    print(f"F1 Score:              {results['f1']:.4f}")
    print(f"Exact Match (EM):      {results['em']:.4f}")
    print(f"Unanswerable Detection: {results['unanswerable_detection_accuracy']:.4f}")
    print(f"\nDataset Statistics:")
    print(f"  Total Samples:       {results['total_samples']}")
    print(f"  Answerable:          {results['answerable_samples']}")
    print(f"  Unanswerable:        {results['unanswerable_samples']}")
    print("="*60)

    # Step 6: Save results
    output_file = 'evaluation_results.json'
    with open(output_file, 'w') as f:
        json.dump({
            'metrics': results,
            'sample_predictions': predictions[:20],  # Save first 20 for inspection
            'config': {
                'embedding_model': EMBEDDING_MODEL,
                'llm_model': LLM_MODEL,
                'chunk_size': CHUNK_SIZE,
                'chunk_overlap': CHUNK_OVERLAP,
                'k_retrieved': K_RETRIEVED,
                'max_samples': MAX_SAMPLES
            }
        }, f, indent=2)

    print(f"\nResults saved to {output_file}")
    print("Evaluation complete!")


In [ ]:
main()

SQuAD v2 RAG Evaluation Pipeline

Step 1: Loading dataset...
Loaded 11873 examples

Step 2: Preparing contexts...
Created 14591 document chunks

Step 3: Setting up RAG system...
Creating new vector store and saving to {persist_directory}...
Created and saved new index!
RAG system ready!

Step 4: Running evaluation...
Using 50 samples for evaluation

Evaluating on 50 examples...


Processing:  66%|██████▌   | 33/50 [01:48<00:53,  3.15s/it]

Error processing example 56dde27d9a695914005b9651: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 200000, Requested 448. Please try again in 3m13.535999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing example 56dde27d9a695914005b9652: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 200000, Requested 632. Please try again in 4m33.024s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing:  70%|███████   | 35/50 [01:48<00:24,  1.61s/it]

Error processing example 5ad3af11604f3c001a3fec63: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 200000, Requested 481. Please try again in 3m27.792s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing example 5ad3af11604f3c001a3fec64: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199999, Requested 482. Please try again in 3m27.792s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


...

Error processing example 5ad3dbc6604f3c001a3ff3ec: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199996, Requested 614. Please try again in 4m23.52s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing example 56de0ffd4396321400ee258d: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199996, Requested 542. Please try again in 3m52.416s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing:  96%|█████████▌| 48/50 [01:50<00:00,  7.94it/s]

Error processing example 56de0ffd4396321400ee258e: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199996, Requested 543. Please try again in 3m52.848s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing example 56de0ffd4396321400ee258f: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199996, Requested 311. Please try again in 2m12.624s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing: 100%|██████████| 50/50 [01:50<00:00,  2.21s/it]

Error processing example 5ad3de8b604f3c001a3ff467: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199995, Requested 546. Please try again in 3m53.712s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing example 5ad3de8b604f3c001a3ff468: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k5stx7hwez8968pj3m4yf370` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 199995, Requested 542. Please try again in 3m51.983999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

EVALUATION RESULTS
F1 Score:              0.1123
Exact Match (EM):      0.3000
Unan